# Desiging recommanding engine that match users with eachother 

In [1]:
# import libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


# set display rows to 1000
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)

In [2]:
data = pd.read_csv('Community.csv')

In [3]:
data.head()

,First Name,Last Name,Gender,Age,Interest
0,Jenna,Murray,Female,18,p
1,Lyndon,Crawford,Male,22,g
2,Vanessa,Hunt,Female,25,e
3,Ted,Barrett,Male,22,l
4,Edwin,Harris,Male,18,j


In [4]:
data= data.reset_index().rename({'index':'id'}, axis =1) 

In [5]:
data.head()

,id,First Name,Last Name,Gender,Age,Interest
0,0,Jenna,Murray,Female,18,p
1,1,Lyndon,Crawford,Male,22,g
2,2,Vanessa,Hunt,Female,25,e
3,3,Ted,Barrett,Male,22,l
4,4,Edwin,Harris,Male,18,j


In [6]:
data.Interest.value_counts()

y    13
w    11
t    11
j    11
c    11
a    10
b    10
x     9
g     9
d     8
h     8
p     8
i     7
l     7
k     7
z     7
u     6
m     6
q     6
e     6
s     6
v     6
f     5
r     4
o     4
n     4
Name: Interest, dtype: int64

In [7]:
interests_dict = {'y': 'Devoloper',
'w': 'Data Science',
'j': 'basketball',
'c': 'Games',
't': 'Hicking',
'b': 'Music',
'a': 'Coffee',
'g': 'Entrepreneurs',
'p': 'Vollyball',
'h': 'Football',
'd': 'Anime',
'z': 'Moveies',
'l': 'Gym',
'i': 'Diet',
'k': 'Self development',
'v': 'History',
'u': 'Religon',
's': 'Travel',
'e': 'Diving',
'q': 'Swimming',
'm': 'Fishing',
'f': 'Hunting',
'o': 'Cars',
'n': 'Techenology',
'r': 'Food',
'x': 'Phtoghraphy'}
interests_list = list(interests_dict.values())

In [8]:
data.Interest = data.Interest.map(interests_dict)

In [9]:
di = {'1':[],'2':[],'3':[],'4':[],'5':[]}

In [10]:
di.values()

dict_values([[], [], [], [], []])

In [11]:
import random

data['Interest2'] = ''
data['Interest3'] = ''
data['Interest4'] = ''
data['Interest5'] = ''

for i in range(200):
    random.seed(i)
    data.loc[i,['Interest','Interest2','Interest3','Interest4','Interest5']] = random.sample(interests_list, 5)

In [12]:
cols = ['Interest','Interest2','Interest3','Interest4','Interest5']
data['combined'] = data[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)

In [13]:
data.sample(10)

,id,First Name,Last Name,Gender,Age,Interest,Interest2,Interest3,Interest4,Interest5,combined
81,81,Eddy,Elliott,Male,20,Religon,Self development,Anime,Travel,Gym,"Religon, Self development, Anime, Travel, Gym"
65,65,Aston,Ross,Male,29,Diet,Football,Religon,Entrepreneurs,Self development,"Diet, Football, Religon, Entrepreneurs, Self d..."
77,77,Garry,Carter,Male,20,Phtoghraphy,Vollyball,Anime,Coffee,Entrepreneurs,"Phtoghraphy, Vollyball, Anime, Coffee, Entrepr..."
131,131,Miranda,Casey,Female,24,Anime,Moveies,Food,Religon,basketball,"Anime, Moveies, Food, Religon, basketball"
171,171,Henry,Brooks,Male,24,Phtoghraphy,Travel,Gym,Diet,Games,"Phtoghraphy, Travel, Gym, Diet, Games"
196,196,Violet,Robinson,Female,28,Fishing,Religon,Cars,Devoloper,Phtoghraphy,"Fishing, Religon, Cars, Devoloper, Phtoghraphy"
28,28,Maria,Perkins,Female,24,Games,Techenology,Hicking,Travel,Swimming,"Games, Techenology, Hicking, Travel, Swimming"
32,32,Cadie,Hall,Female,19,basketball,Coffee,Hicking,Football,Cars,"basketball, Coffee, Hicking, Football, Cars"
161,161,William,Harris,Male,22,Religon,Coffee,Travel,Music,Entrepreneurs,"Religon, Coffee, Travel, Music, Entrepreneurs"
154,154,Eleanor,Ferguson,Female,22,Coffee,Phtoghraphy,Gym,History,Self development,"Coffee, Phtoghraphy, Gym, History, Self develo..."


In [14]:
# implement recommanding system (C)

In [15]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [17]:
community_tfidf = TfidfVectorizer()
community_count = CountVectorizer()

In [18]:
community_description_matrix = community_tfidf.fit_transform(data['combined'])
community_description_matrix = community_count.fit_transform(data['combined'])

In [19]:
community_description_matrix.shape

(200, 28)

In [20]:
community_description_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 1]], dtype=int64)

In [21]:
cosine_similarity = linear_kernel(community_description_matrix, community_description_matrix)

In [22]:
indices = pd.Series(data[['Interest','Interest2','Interest3','Interest4','Interest5']].index)

In [23]:
def recommend(index, cosine_sim=cosine_similarity):
    id = indices[index]
    
    # Get the pairwsie similarity scores of all books compared to that book, 
    # sorting them and getting top 5
    
    similarity_scores = list(enumerate(cosine_sim[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]

    # Get the books index
    community_index = [i[0] for i in similarity_scores]

    # Return the top 5 most similar books using integer-location based indexing (iloc)
    return data.iloc[community_index]

In [24]:
recommend(10)

,id,First Name,Last Name,Gender,Age,Interest,Interest2,Interest3,Interest4,Interest5,combined
98,98,Paige,Parker,Female,27,Moveies,Diving,Data Science,Diet,History,"Moveies, Diving, Data Science, Diet, History"
136,136,Alford,Montgomery,Male,21,Diving,Diet,Religon,Data Science,History,"Diving, Diet, Religon, Data Science, History"
57,57,Reid,Williams,Male,29,Data Science,Moveies,Diving,Swimming,Devoloper,"Data Science, Moveies, Diving, Swimming, Devol..."
75,75,Isabella,Murphy,Female,30,Self development,Diving,Diet,Data Science,Techenology,"Self development, Diving, Diet, Data Science, ..."
127,127,Mike,Hawkins,Male,24,Data Science,Swimming,Diving,Games,Devoloper,"Data Science, Swimming, Diving, Games, Devoloper"
